In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()                   #input -? OUtput? RF
        self.conv1 = nn.Conv2d(1, 8, 3, bias = False) #28-26
        self.conv2 = nn.Conv2d(8, 16, 3, bias = False) #26-24
        self.conv3 = nn.Conv2d(16, 32, 3, bias = False) #24-22
        self.pool1 = nn.MaxPool2d(2, 2) #22-11
        self.conv4 = nn.Conv2d(32, 8, 1, bias = False) #11-11
        self.conv5 = nn.Conv2d(8, 16, 3, bias = False) #11-9
        self.conv6 = nn.Conv2d(16, 16, 3, bias = False) #9-7
        self.conv7 = nn.Conv2d(16, 32, 3, bias = False) #7-5
        self.conv8 = nn.Conv2d(32, 10, 3, bias = False) #5-3
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.bn81 = nn.BatchNorm2d(8)
        self.bn82 = nn.BatchNorm2d(8)
        self.bn161 = nn.BatchNorm2d(16)
        self.bn162 = nn.BatchNorm2d(16)
        self.bn163 = nn.BatchNorm2d(16)
        self.bn321 = nn.BatchNorm2d(32)
        self.bn322 = nn.BatchNorm2d(32)
        self.drop = nn.Dropout(0.05)

    def forward(self, x):
        # x = self.pool1(F.relu(self.conv3(F.relu(self.conv2(F.relu(self.conv1(x)))))))
        # x = F.relu(self.conv7(F.relu(self.conv6(F.relu(self.conv5(F.relu(self.conv4(x))))))))
        # x =self.conv7(F.relu(self.conv6((F.relu(self.conv5(x))))))
        x = self.drop(self.bn81(F.relu(self.conv1(x))))
        x = self.drop(self.bn161(F.relu(self.conv2(x))))

        x = self.drop(self.bn321(F.relu(self.conv3(x))))
        x = self.pool1(x)
        x = self.drop(self.bn82(F.relu(self.conv4(x))))
        x = self.drop(self.bn162(F.relu(self.conv5(x))))
        x = self.drop(self.bn163(F.relu(self.conv6(x))))
        x = self.drop(self.bn322(F.relu(self.conv7(x))))
        x = self.conv8(x)
        x = self.avgpool(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
       BatchNorm2d-2            [-1, 8, 26, 26]              16
           Dropout-3            [-1, 8, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           1,152
       BatchNorm2d-5           [-1, 16, 24, 24]              32
           Dropout-6           [-1, 16, 24, 24]               0
            Conv2d-7           [-1, 32, 22, 22]           4,608
       BatchNorm2d-8           [-1, 32, 22, 22]              64
           Dropout-9           [-1, 32, 22, 22]               0
        MaxPool2d-10           [-1, 32, 11, 11]               0
           Conv2d-11            [-1, 8, 11, 11]             256
      BatchNorm2d-12            [-1, 8, 11, 11]              16
          Dropout-13            [-1, 8, 11, 11]               0
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)



Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm

def train_eval(model, device, train_loader):
    model.eval()
    train_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            train_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    train_loss /= len(train_loader.dataset)

    print('\n\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))
  
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
    train_eval(model, device, train_loader)


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
epochs = 20
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(epochs):
    print("Epoch: {}/{}..\n ".format(epoch+1, epochs));
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

Epoch: 1/20..
 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.06170421838760376 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 41.38it/s]




Train set: Average loss: 0.0582, Accuracy: 58933/60000 (98.222%)



  0%|          | 0/469 [00:00<?, ?it/s]

...



Train set: Average loss: 0.0091, Accuracy: 59843/60000 (99.738%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0216, Accuracy: 9938/10000 (99.380%)

Epoch: 20/20..
 


loss=0.0006616115570068359 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 42.28it/s]




Train set: Average loss: 0.0086, Accuracy: 59851/60000 (99.752%)


Test set: Average loss: 0.0198, Accuracy: 9938/10000 (99.380%)

